### Data Preprocessing and Inspection
#### Structure of this notebook:  
    **Aggregating the data into a dataset:** 
      
    A. Preprocessing  
      
    **Exploring the data and descriptive statistics:**   
      
    B. Understanding the Likers & Retweeters datasets  
    C. Understanding script performance
    D. Understanding user activity

In [ ]:
import os
import glob
import json
import pandas as pd
import csv
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import itertools
from matplotlib import pyplot as plt
from collections import Counter
from ast import literal_eval 
from aux import *

## A. Preprocessing 
(To be run once only)

#### Prelims and aggregating all likers/retweeters files including the final harvest data into like/retweet-tweet matrices.   
#### Where likers/retweeters are listed in columns, and tweet-IDs in rows.  
#### Entry[tweet_i,liker_j] evaluates to 1, if liker_j has liked tweet_i.

In [ ]:
#TODO: What is your pull-folder called?
my_pull = ' ' # pull folder name: Pull-DD-MM-YYY-hour:minute:second

In [ ]:
# list all likers/retweeters files
# TODO: adjust path to your pull-folder! 
filenames_l = glob.glob(os.path.join(my_pull,'CSVs/Likers_of_alarms/*ikers*')) 
filenames_r = glob.glob(os.path.join(my_pull,'CSVs/Retweeters_of_alarms/*etweeters*'))
# Import all csvs and convert to pkl files
# likers:
for file in filenames_l:
    likers = pd.read_csv(file, converters={"likers": literal_eval})
    file = file.replace("csv","pkl")
    likers.to_pickle(file)
# retweeters:
for file in filenames_r:
    retweeters = pd.read_csv(file, converters={"retweeters": literal_eval})
    file = file.replace("csv","pkl")
    retweeters.to_pickle(file)
# load all pkl file names
pkl_l = glob.glob(os.path.join(my_pull,'CSVs/Likers_of_alarms/*pkl')) # all pkl file names 
pkl_r = glob.glob(os.path.join(my_pull,'CSVs/Retweeters_of_alarms/*pkl')) # all pkl file names 

In [ ]:
# read pkl files, set tweet id as index (makes data handling and indexing way easier), make vote dataframe
# Aggregate votes in dataframe: tweet ID as row index, user names as column headings
likers_incomplete = pd.DataFrame()

for files in pkl_l:

    df = pd.read_pickle(files)
    df.set_index('Unnamed: 0', inplace = True)
    
    for tweet in df.index:      
            for user in df.at[tweet,'likers']:
                likers_incomplete.at[tweet, user] = 1 

In [ ]:
# same as above for retweeters
retweeters_incomplete = pd.DataFrame()

for files in pkl_r:

    df = pd.read_pickle(files)
    df.set_index('Unnamed: 0', inplace = True)
    
    for tweet in df.index:      
            for user in df.at[tweet,'retweeters']:
                retweeters_incomplete.at[tweet, user] = 1 

#### Add final harvest data to dataframe

In [ ]:
# likes
finalharvest_l = pd.read_pickle(os.path.join(my_pull,'likers_final_harvest_complete.pkl')) # We use a file to keep track of the log so we can remember position if we need to restart due to error

In [ ]:
# retweets
finalharvest_r = pd.read_pickle(os.path.join(my_pull,'retweeters_final_harvest_complete.pkl')) # We use a file to keep track of the log so we can remember position if we need to restart due to error

In [ ]:
# add final harvest to dataframe
for tweet in finalharvest_l.index:      
    for user in finalharvest_l.at[tweet,'likers']:
                likers_incomplete.at[tweet, user] = 1 

In [ ]:
# As above for retweets            
for tweet in finalharvest_r.index:      
    for user in finalharvest_r.at[tweet,'retweeters']:
                retweeters_incomplete.at[tweet, user] = 1 

In [ ]:
likers_complete = likers_incomplete
retweeters_complete = retweeters_incomplete

#### Save complete datasets: likers and retweeters

In [ ]:
# Now dataset complete, save as csv so it can also be passed on to data analysis
# TODO: name file
likers_complete.to_csv('like_votes_with_final_harvest_XXX.csv')
retweeters_complete.to_csv('retweets_votes_with_final_harvest_XXX.csv')

## B. Understanding the Likers & Retweeters datasets: Exploring the data/descriptive statistics

Load the data:

In [ ]:
# If A. was run, then load data
likers_complete = pd.read_csv('like_votes_with_final_harvest_XXX.csv')
pd.options.display.float_format = '{:.2f}'.format
likers_complete.set_index('Unnamed: 0', inplace = True)
likers_complete.index.names = ['tweet']

retweeters_complete = pd.read_csv('retweets_votes_with_final_harvest_XXX.csv')
pd.options.display.float_format = '{:.2f}'.format
retweeters_complete.set_index('Unnamed: 0', inplace = True)
retweeters_complete.index.names = ['tweet']

finalharvest_l = pd.read_pickle(os.path.join(my_pull,'likers_final_harvest_complete.pkl'))
finalharvest_r = pd.read_pickle(os.path.join(my_pull,'retweeters_final_harvest_complete.pkl'))
finalharvest_l.index.names = ['tweet']
finalharvest_r.index.names = ['tweet']

In [ ]:
# if pkl:
#likers = pd.read_pickle(os.path.join(my_pull,'binary-matrix-likers.pkl'))
#retweeters = pd.read_pickle(os.path.join(my_pull,'binary-matrix-retweeters.pkl'))
#likers_complete = likers
#retweeters_complete = retweeters
#finalharvest_l = pd.read_pickle(os.path.join(my_pull,'likers_final_harvest_complete.pkl'))
#finalharvest_r = pd.read_pickle(os.path.join(my_pull,'retweeters_final_harvest_complete.pkl'))
#finalharvest_l.index.names = ['tweet']
#finalharvest_r.index.names = ['tweet']

Let us look at the dataset of liking users, stored in  `likers`. In `likers`, the row index is tweet ID and the column names are user names. A cell contains `1` if the user liked the tweet, else `NaN`. Both rows and columns are sorted: rows numerically, columns alphabetically. The `retweeters` and `finalharvest_` dataframes are structured in the same way.

The next cells are examples of the following:

1. As the index is the tweet ID, we can look up a single tweet by using its ID, using `.loc`.
2. To find some tweet IDs, we may want to look up at the subsection of the first 3 tweets and the first 5 users, using `.iloc`.
3. TODO: List of linking users of a single tweet.

In [ ]:
likers.loc[1537712147500781569]

In [ ]:
likers.iloc[0:3,0:5]

## Summary Numbers

In [ ]:
totals = pd.DataFrame()
totals.loc[1,'Tweets Liked'] = likers.shape[0]
totals.loc[1,'Likers'] = likers.shape[1]
totals.loc[1,'Likes'] = sum(likers.sum(axis = 1, skipna = True))
#totals.loc[1,'Sanity Check 1'] = sum(likers.sum(axis = 0, skipna = True))
totals.loc[1,'Tweets Retweeted'] = retweeters.shape[0]
totals.loc[1,'Retweeters'] = retweeters.shape[1]
totals.loc[1,'Retweets'] = sum(retweeters.sum(axis = 1, skipna = True))
#totals.loc[1,'Sanity Check 2'] = sum(retweeters.sum(axis = 0, skipna = True))
totals

## C. Understanding script performance
### How many likers/retweeters did the script curate? We are looking at a dataset comprising those tweets that were also one last time collected in the final harvest

In [ ]:
# Optional: How many tweets got <my_likersAtLeast> likes? How many tweets got <my_retweetersAtLeast> retweets?
# see parameters my_likersAtLeast/my_retweetersAtLeast
my_likersAtLeast = 10 # TODO SET YOUR PARAMTER HERE
my_retweetersAtLeast = 3 # TODO SET YOUR PARAMETER HERE

Atleast = pd.DataFrame()
Atleast.loc[1, 'All tweets liked'] = likers.shape[0]
Atleast.loc[1,'Tweets with my_likersAtLeast'] = sum((likers.sum(axis = 1, skipna = True)) >= my_likersAtLeast) 
Atleast.loc[1, 'All tweets retweeted'] = retweeters.shape[0]
Atleast.loc[1,'Tweets with my_retweetersAtLeast'] = sum((retweeters.sum(axis = 1, skipna = True)) >= my_retweetersAtLeast) 
Atleast

In [ ]:
subset_likerscomplete = pd.merge(likers_complete, finalharvest_l, left_index=True, right_index=True)
subset_retweeterscomplete = pd.merge(retweeters_complete, finalharvest_r, left_index=True, right_index=True)

In [ ]:
# like count at time of final harvest
likecount = finalharvest_l['like_count']
# number of collected likers 
likerscollected = subset_likerscomplete.sum(axis = 1, skipna = True) 
# retweet count at time of final harvest
retweetcount = finalharvest_r['retweet_count']
# number of collected retweeters
retweeterscollected = subset_retweeterscomplete.sum(axis = 1, skipna = True) 

In [ ]:
# Absolute number of missed likes/retweets per tweet
plot_missed(likecount, likerscollected, retweetcount, retweeterscollected)

In [ ]:
# Share of missed likes/retweets given total of received likes/retweets per tweet
plot_missed_relative(likecount, likerscollected, retweetcount, retweeterscollected)

In [ ]:
# Supplemented with total number of likes/retweets each tweet attracted: 
plot_missed_relative_absolutecount(likecount, likerscollected, retweetcount, retweeterscollected)


In [ ]:
# inspect numbers more closely: likers
d = {'collected likers': likerscollected, 'likecount': likecount, 'difference': likecount-likerscollected, 'percent': ((likecount-likerscollected)/likecount)}
inspectlikes = pd.DataFrame(data=d)
inspectlikes

In [ ]:
# inspect numbers more closely: retweeteres
d = {'collected retweeters': retweeterscollected, 'retweetcount': retweetcount, 'difference': retweetcount-retweeterscollected, 'percent': ((retweetcount-retweeterscollected)/retweetcount)}
inspectretweets = pd.DataFrame(data=d)
inspectretweets

In [ ]:
perf = pd.DataFrame()
perf.loc[1, '% tweets with 10 or more too many (deleted):'] = round(len(inspectlikes[inspectlikes['difference'] <-10])/len(inspectlikes),4)
perf.loc[2,'% tweets with 10 or more too many (deleted):'] = round(len(inspectretweets[inspectretweets['difference'] <-10])/len(inspectretweets), 4)

perf.loc[1, '% tweets with 10 or more missed:'] = round(len(inspectlikes[inspectlikes['difference'] >10])/len(inspectlikes),4)
perf.loc[2, '% tweets with 10 or more missed:'] = round(len(inspectretweets[inspectretweets['difference'] >10])/len(inspectretweets),4)

perf.loc[1, '% tweets with 10% or more too many (deleted):'] = round(len(inspectlikes[inspectlikes['percent'] <-.1])/len(inspectlikes),4)
perf.loc[2,'% tweets with 10% or more too many (deleted):'] = round(len(inspectretweets[inspectretweets['percent'] >.1] )/len(inspectretweets),4)


perf.loc[1, '% tweets with 10% or more missed:'] = round(len(inspectlikes[inspectlikes['percent'] >.1])/len(inspectlikes),4)
perf.loc[2, '% tweets with 10% or more missed:'] = round(len(inspectretweets[inspectretweets['percent'] <-.1] )/len(inspectretweets),4)

perf.loc[1, '% tweets with complete:'] = round(len(inspectlikes[inspectlikes['difference'] == 0])/len(inspectlikes),4)
perf.loc[2, '% tweets with complete:'] = round(len(inspectretweets[inspectretweets['difference'] == 0])/len(inspectretweets),4)


perf.index = ['Likes', 'Retweets']

perf

In [ ]:
# Inspect (highly popular) tweets in terms of like count
likecount

In [ ]:
# Inspect (highly popular) tweets in terms of retweet count
retweetcount

## D. Understanding user activity
How many likes/retweets did the users place? How many unique likers/retweeters are in the dataset? 

### How many likers/retweeters did the script curate? Dataset comprising those tweets that were one last time collected in the final harvest

In [ ]:
freqtables_l, freqtable_r = make_frequency_table(likers_complete, retweeters_complete)

In [ ]:
freqtables_l.head()

In [ ]:
freqtable_r.head()

In [ ]:
plot_frequency(freqtable_l, freqtable_r)

In [ ]:
users = pd.DataFrame()

users.loc[1, 'users placed more than 1:'] = freqtable_l.loc[freqtable_l['placedlikes'] > 1, 'freqlikers'].sum()
users.loc[2,'users placed more than 1:'] = freqtable_r.loc[freqtable_r['placedretweets'] > 1, 'freqretweeters'].sum()

users.loc[1, '% users placed more than 1:'] = round((freqtable_l.loc[freqtable_l['placedlikes'] > 1, 'freqlikers'].sum())/sum(freqtable_l['freqlikers']),4)
users.loc[2, '% users placed more than 1:'] = round((freqtable_r.loc[freqtable_r['placedretweets'] > 1, 'freqretweeters'].sum())/sum(freqtable_r['freqretweeters']),4)

users.loc[1, 'users placed more than 2:'] = freqtable_l.loc[freqtable_l['placedlikes'] > 2, 'freqlikers'].sum()
users.loc[2,'users placed more than 2:'] = freqtable_r.loc[freqtable_r['placedretweets'] > 2, 'freqretweeters'].sum()

users.loc[1, '% users placed more than 2:'] = round((freqtable_l.loc[freqtable_l['placedlikes'] > 2, 'freqlikers'].sum())/sum(freqtable_l['freqlikers']),4)
users.loc[2, '% users placed more than 2:'] = round((freqtable_r.loc[freqtable_r['placedretweets'] > 2, 'freqretweeters'].sum())/sum(freqtable_r['freqretweeters']),4)

users.loc[1, 'users placed more than 3:'] = freqtable_l.loc[freqtable_l['placedlikes'] > 3, 'freqlikers'].sum()
users.loc[2,'users placed more than 3:'] = freqtable_r.loc[freqtable_r['placedretweets'] > 3, 'freqretweeters'].sum()

users.loc[1, '% users placed more than 3:'] = round((freqtable_l.loc[freqtable_l['placedlikes'] > 3, 'freqlikers'].sum())/sum(freqtable_l['freqlikers']),4)
users.loc[2, '% users placed more than 3:'] = round((freqtable_r.loc[freqtable_r['placedretweets'] > 3, 'freqretweeters'].sum())/sum(freqtable_r['freqretweeters']),4)

users.loc[1, 'users placed more than 4:'] = freqtable_l.loc[freqtable_l['placedlikes'] > 4, 'freqlikers'].sum()
users.loc[2,'users placed more than 4:'] = freqtable_r.loc[freqtable_r['placedretweets'] > 4, 'freqretweeters'].sum()

users.loc[1, '% users placed more than 4:'] = round((freqtable_l.loc[freqtable_l['placedlikes'] > 4, 'freqlikers'].sum())/sum(freqtable_l['freqlikers']),4)
users.loc[2, '% users placed more than 4:'] = round((freqtable_r.loc[freqtable_r['placedretweets'] > 4, 'freqretweeters'].sum())/sum(freqtable_r['freqretweeters']),4)

users.loc[1, 'users placed more than 50:'] = freqtable_l.loc[freqtable_l['placedlikes'] > 50, 'freqlikers'].sum()
users.loc[2,'users placed more than 50:'] = freqtable_r.loc[freqtable_r['placedretweets'] > 50, 'freqretweeters'].sum()

users.loc[1, '% users placed more than 50:'] = round((freqtable_l.loc[freqtable_l['placedlikes'] > 50, 'freqlikers'].sum())/sum(freqtable_l['freqlikers']),4)
users.loc[2, '% users placed more than 50:'] = round((freqtable_r.loc[freqtable_r['placedretweets'] > 50, 'freqretweeters'].sum())/sum(freqtable_r['freqretweeters']),4)


users.index = ['Likes', 'Retweets']
users
